# Building GPT on russian poetry

In [48]:
# Make imports
import torch
import math
from google.colab import files

In [13]:
# Prepare dataset
!wget https://github.com/gromdimon/ml-random/raw/master/pushgpt/data/data.txt

--2024-01-11 08:01:02--  https://github.com/gromdimon/ml-random/raw/master/pushgpt/data/data.txt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/gromdimon/ml-random/master/pushgpt/data/data.txt [following]
--2024-01-11 08:01:02--  https://raw.githubusercontent.com/gromdimon/ml-random/master/pushgpt/data/data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2650695 (2.5M) [text/plain]
Saving to: ‘data.txt’

data.txt            100%[===================>]   2.53M  --.-KB/s    in 0.06s   

2024-01-11 08:01:03 (39.1 MB/s) - ‘data.txt’ saved [2650695/2650695]



In [14]:
with open('data.txt', 'r') as f:
  text = f.read()

print(f'Length of text: {len(text)}')

Length of text: 1495941


In [15]:
print(text[:300])

Ты никогда меня, Алина, не любила.
Не виновата ты - я сам обманывал себя! 
Надежда тщетная мне льстила, 
Когда украдкою смотрел я на тебя! 
Ты говорила - мне казалось, 
Что в смысле слов твоих есть тайный смысл любви; 
Я думал так, когда в мечты твои
Моя случайно прививалась; 
Так, словом: каждый ша


In [16]:
# unique chars
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(f'Vocabulary size: {vocab_size}')

	
 !"#$%'()*+,-./0123456789:;<=>?ABCDEFGHIJKLMNPQRSTUVY[\]_abcdefghijklmnopqrstuvwxyz{}~¤«»́;АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЫЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяёө‎–—―“„…﻿
Vocabulary size: 165


In [17]:
# Encoding
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]   # encoder: take a string and return numerical values
decode = lambda l: ''.join([itos[i] for i in l])   # decoder: take a list of integers and return a corresponding string

print(encode("Здравствуй, мир!"))
print(decode(encode("Здравствуй, мир!")))

[100, 127, 139, 123, 125, 140, 141, 125, 142, 132, 13, 2, 135, 131, 139, 3]
Здравствуй, мир!


In [19]:
# Encode entire text and store as torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1495941]) torch.int64
tensor([111, 150,   2, 136, 131, 133, 137, 126, 127, 123,   2, 135, 128, 136,
        154,  13,   2,  93, 134, 131, 136, 123,  13,   2, 136, 128,   2, 134,
        153, 124, 131, 134, 123,  15,   1, 106, 128,   2, 125, 131, 136, 137,
        125, 123, 141, 123,   2, 141, 150,   2,  14,   2, 154,   2, 140, 123,
        135,   2, 137, 124, 135, 123, 136, 150, 125, 123, 134,   2, 140, 128,
        124, 154,   3,   2,   1, 106, 123, 127, 128, 129, 127, 123,   2, 141,
        148, 128, 141, 136, 123, 154,   2, 135, 136, 128,   2, 134, 151, 140,
        141, 131, 134, 123,  13,   2,   1, 103, 137, 126, 127, 123,   2, 142,
        133, 139, 123, 127, 133, 137, 153,   2, 140, 135, 137, 141, 139, 128,
        134,   2, 154,   2, 136, 123,   2, 141, 128, 124, 154,   3,   2,   1,
        111, 150,   2, 126, 137, 125, 137, 139, 131, 134, 123,   2,  14,   2,
        135, 136, 128,   2, 133, 123, 130, 123, 134, 137, 140, 151,  13,   2,
          1, 116, 141, 137,   

In [20]:
# Split data for training and validation
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [23]:
block_size = 10
train_data[:block_size+1]

tensor([111, 150,   2, 136, 131, 133, 137, 126, 127, 123,   2])

In [24]:
# Representation of what transformer internally should do
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([111]) the target: 150
when input is tensor([111, 150]) the target: 2
when input is tensor([111, 150,   2]) the target: 136
when input is tensor([111, 150,   2, 136]) the target: 131
when input is tensor([111, 150,   2, 136, 131]) the target: 133
when input is tensor([111, 150,   2, 136, 131, 133]) the target: 137
when input is tensor([111, 150,   2, 136, 131, 133, 137]) the target: 126
when input is tensor([111, 150,   2, 136, 131, 133, 137, 126]) the target: 127
when input is tensor([111, 150,   2, 136, 131, 133, 137, 126, 127]) the target: 123
when input is tensor([111, 150,   2, 136, 131, 133, 137, 126, 127, 123]) the target: 2


In [27]:
ix = torch.randint(len(data) - 10, (4,))
print(ix)
x = torch.stack([data[i:i+10] for i in ix])
y = torch.stack([data[i+1:i+10+1] for i in ix])
print(x)
print(y)

tensor([1425684, 1321777,  839704,  852361])
tensor([[134, 131,   2, 128, 126, 137,   1,  95, 139, 123],
        [138, 128, 146, 123, 134, 151,   2, 140, 125, 137],
        [134, 123, 141, 151,  32,   2, 146, 141, 137,   2],
        [ 27,   2,  95, 128, 139, 128, 136, 131, 145, 123]])
tensor([[131,   2, 128, 126, 137,   1,  95, 139, 123, 129],
        [128, 146, 123, 134, 151,   2, 140, 125, 137, 153],
        [123, 141, 151,  32,   2, 146, 141, 137,   2, 135],
        [  2,  95, 128, 139, 128, 136, 131, 145, 123,   1]])


In [28]:
torch.manual_seed(142)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 10 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 10])
tensor([[128, 124, 154,   2, 141, 139, 128, 125, 137, 129],
        [139, 137, 127, 128, 141, 128, 134, 151,  27,   1],
        [141, 131,   2, 135, 139, 123, 146, 136, 137, 132],
        [138, 137, 127, 139, 142, 126,  32,   1, 106, 128]])
targets:
torch.Size([4, 10])
tensor([[124, 154,   2, 141, 139, 128, 125, 137, 129, 131],
        [137, 127, 128, 141, 128, 134, 151,  27,   1, 112],
        [131,   2, 135, 139, 123, 146, 136, 137, 132,   2],
        [137, 127, 139, 142, 126,  32,   1, 106, 128,   2]])
----
when input is [128] the target: 124
when input is [128, 124] the target: 154
when input is [128, 124, 154] the target: 2
when input is [128, 124, 154, 2] the target: 141
when input is [128, 124, 154, 2, 141] the target: 139
when input is [128, 124, 154, 2, 141, 139] the target: 128
when input is [128, 124, 154, 2, 141, 139, 128] the target: 125
when input is [128, 124, 154, 2, 141, 139, 128, 125] the target: 137
when input is [128, 124, 154, 2, 141, 13

## Bigram Language Model

In [29]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(142)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([40, 165])
tensor(5.5798, grad_fn=<NllLossBackward0>)
	уg-ГоъиgCсёT“gЦ	еAохт	мОШрөТm4fjИ‎Р—0Q!;¤дй «т‎оP<щm\Н8{7–бc0zlфR»gё"8к xAис)Лдь,ЯРор—6y_Фыж1m\lФыy7


In [30]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [35]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())
print(f'Expected random nll: {-math.log(1/vocab_size)}')


5.1023149490356445
Expected random nll: 5.10594547390058


In [36]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

	ТbЖЕъgsHoо9?oе2n4мo,яrц!QpP)6JjS{ХeраЙ0	?g7со9ЗFNcИС…Г<аэ0кaУЛs=LKЭАf86E}a)П,¤kГпъfG¤ВМ*8hАшк—ёNАнвh*ЖЧ―Pd»	%У}ялPbЖСд6к84Я1LюAмh(Uвi*ёДоr;Ы.б3xKыИз8Хвлm~л_>8ЯгШОУ
	„oVEwВЩВP)иAй9GРC+црbPLTэe<gЦХe„ihЧ?Йд―bмзрЛВ7Зl(Д8о“сBТp#teэ„Йиьи;
—ЩyLМaq_R,БGө в5т0,;ЧuфtВСуЩt―;Yй  vl﻿́2k0‎ЗL>b-lx
MF_{aBstPsЦCЮnяNА«НhР‎уЗw вCS‎8В E“—ьM3)	7.―в'=ХJгR…"Щ}KM)<щйC+BУ+у-2
	г
–5N?;')–joщеCбf;s=6e]А«+-\;.BТ{<m#ыjYВ/7ЭGЩ﻿	б-bюөGnUikQCQхoщйЕ?йгvТ„о:ыЫЮ-—…ёъRJj+bрФ!7тr:33KrЦlЮ9НОCоSвй.2Ду;PH/ЭA;n~HнФжю	Г﻿;S_ёщг_КqөгH~т?р


## Self-attention construction


In [37]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [38]:
# consider the following toy example:

torch.manual_seed(142)
B,T,C = 4,10,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 10, 2])

In [39]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [40]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [41]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


True

In [42]:
# version 4: self-attention!
torch.manual_seed(142)
B,T,C = 4,10,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 10, 16])

In [43]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.4103, 0.5897, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.3415, 0.6370, 0.0215, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.1249, 0.0940, 0.1776, 0.6035, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0870, 0.1354, 0.6032, 0.1005, 0.0739, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0431, 0.0233, 0.7775, 0.0847, 0.0605, 0.0109, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0144, 0.0109, 0.0040, 0.0890, 0.8487, 0.0304, 0.0026, 0.0000, 0.0000,
         0.0000],
        [0.0647, 0.0102, 0.5750, 0.0363, 0.0015, 0.0026, 0.3072, 0.0024, 0.0000,
         0.0000],
        [0.1009, 0.3238, 0.0078, 0.0294, 0.0263, 0.3270, 0.1144, 0.0251, 0.0453,
         0.0000],
        [0.0090, 0.0381, 0.0023, 0.0043, 0.1291, 0.0902, 0.0045, 0.1382, 0.5279,
         0.0564]], grad_fn=<

In [44]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(142)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [45]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.0734), tensor(1.0049))

In [46]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(6.5565e-09), tensor(1.0000))

## Actual Model

In [ ]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 8000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 8
n_layer = 8
dropout = 0.0
# ------------

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

0.423845 M parameters
step 0: train loss 5.2048, val loss 5.2012
step 100: train loss 2.8167, val loss 2.7798
step 200: train loss 2.6767, val loss 2.6694
step 300: train loss 2.6230, val loss 2.6166
step 400: train loss 2.5753, val loss 2.5711
step 500: train loss 2.5177, val loss 2.5023
step 600: train loss 2.4548, val loss 2.4574


### Results:
1 - batch_size=16, block_size=32, max_iters=5000, n_embd=32, n_head=4, n_layer=4

2 - batch_size=32, block_size=64, max_iters=8000, n_embd=64, n_head=8, n_layer=8

| 1 | 2 |
| --- | --- |
| step 0: train loss 5.2444, val loss 5.2621 | step 0: train loss 5.2048, val loss 5.2012 |
| step 100: train loss 2.9265, val loss 2.8814 | step 100: train loss 2.8167, val loss 2.7798 |
| step 4999: train loss 2.0749, val loss 2.0706 | --- |



In [49]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

	" А исчах ночесился,
И рекаетой ялося.У

Прогверенальная домнего ужастный:
Моловыхнуюга
Скоры, душал дить?
Во зей гна всетсина тошной
Не мотерчистом ян на дорушать,
Вольмом тучески мне 
Собшнылой почтетн,
Не всюди во скруг и меня гманьстыхать голмок?

От сfуть глюбу: и бурий поны
Кров, тоборилсева родали путы ты буйном,
Путный одешь, нильный бль, носль кела
Замой к себет кракалый. 
Увет:
И я покойть не на разликоговой —
Ны бирок, гтоб приземраживнием бемчись в —
Тамого слеuчистно сон погучи
Скрось леннуться вот каторв,
Вердце учем, суде стрыйну пучнала,
Я крыласка был тих звы,
Ракею! ровся дру можится,
Цваю милый - блажен 
И ―ежный - прозыл
Покого волныз обчал на,
Раз дику…И хорилый, свершные койдети?
Гот, милый нестрыта ндадлии?
И замило когройки корькая;
Старь в ввиды встрать прищари - — голлость пренян,
И как слебой крыты слачй на покопыть,
С отмой илот.

Под свозыт гое - наей.

Вот круг нечных за севей Керрануть -
Бузненно вермыв вигу,
Эточелося пригоробь!
Вхорга весоранна в на уг

In [53]:
!mkdir model
torch.save(m.state_dict(), 'model/checkpoint.pt')

# download checkpoint file
files.download('model/checkpoint.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [52]:
# Example model load
state_dict = torch.load('model/checkpoint.pt')
print(state_dict.keys())
model = BigramLanguageModel()
model.load_state_dict(state_dict)

# generate
context = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(context, max_new_tokens=2000)[0].tolist()))

odict_keys(['token_embedding_table.weight', 'position_embedding_table.weight', 'blocks.0.sa.heads.0.tril', 'blocks.0.sa.heads.0.key.weight', 'blocks.0.sa.heads.0.query.weight', 'blocks.0.sa.heads.0.value.weight', 'blocks.0.sa.heads.1.tril', 'blocks.0.sa.heads.1.key.weight', 'blocks.0.sa.heads.1.query.weight', 'blocks.0.sa.heads.1.value.weight', 'blocks.0.sa.heads.2.tril', 'blocks.0.sa.heads.2.key.weight', 'blocks.0.sa.heads.2.query.weight', 'blocks.0.sa.heads.2.value.weight', 'blocks.0.sa.heads.3.tril', 'blocks.0.sa.heads.3.key.weight', 'blocks.0.sa.heads.3.query.weight', 'blocks.0.sa.heads.3.value.weight', 'blocks.0.sa.proj.weight', 'blocks.0.sa.proj.bias', 'blocks.0.ffwd.net.0.weight', 'blocks.0.ffwd.net.0.bias', 'blocks.0.ffwd.net.2.weight', 'blocks.0.ffwd.net.2.bias', 'blocks.0.ln1.weight', 'blocks.0.ln1.bias', 'blocks.0.ln2.weight', 'blocks.0.ln2.bias', 'blocks.1.sa.heads.0.tril', 'blocks.1.sa.heads.0.key.weight', 'blocks.1.sa.heads.0.query.weight', 'blocks.1.sa.heads.0.value.weig